# 1. Load synthetic notes

In [ ]:
import pandas as pd
import json

df = pd.read_csv("data/synthetic_notes.csv")

# 2. Extract data from our notes
Code for [I Ran OpenAI’s New Open Model on My Laptop to Extract Medical Data — Here’s What Happened](https://medium.com/towards-artificial-intelligence/i-ran-openais-new-open-model-on-my-laptop-to-extract-medical-data-here-s-what-happened-aeb6acddfede)

Updated for [New article]()

In [3]:
from discharge_agent.extractions.extraction import run_extraction_with_json_evaluation

# Updated extraction to run on the full dataframe with retries if JSON format is not correct

results, summary = run_extraction_with_json_evaluation(df, text_col="note_text", max_retries=3)

Note 5 attempt 1 failed: Extra data: line 1 column 1204 (char 1203)
Note 6 attempt 1 failed: Extra data: line 1 column 1265 (char 1264)

=== Extraction Evaluation ===
Processed: 10
Valid JSON: 10
Invalid JSON (after retries): 0
Success rate: 100.0%


## Format the output for the tool use evaluation

In [2]:
from discharge_agent.llm.prompts import get_messages
import json
# Create dataset for evaluation
with open("data/processed_notes/tool_evaluation_samples_v2.jsonl", "w") as f:
    for result in results:
        messages = get_messages(result)
        case_sample = {
            "system": messages[0]['content'],
            "user": messages[1]['content'],
            "result_json": result
        }
        f.write(json.dumps(case_sample) + "\n")


# 3. Run discharge checks
Demo code for [I Built a Local Clinical AI Agent from Scratch - Here's How]()

In [1]:
import json

with open("data/processed_notes/upper_gi_bleed.json", "r") as f:
    result_json = json.load(f)

In [5]:
from discharge_agent.pipelines.discharge_checker import check_discharge_safety
from discharge_agent.llm.prompts import get_messages

messages = get_messages(result_json)
final = check_discharge_safety(messages)

{'function': {'name': 'flag_labs', 'arguments': {'labs': [{'date': '2024-08-24', 'name': 'Hgb', 'value': '10.2'}, {'date': '2024-08-24', 'name': 'Hct', 'value': '31%'}, {'date': '2024-08-24', 'name': 'Plt', 'value': '245'}, {'date': '2024-08-24', 'name': 'BUN', 'value': '25'}, {'date': '2024-08-24', 'name': 'Cr', 'value': '1.1'}]}}}
{'function': {'name': 'umls_normalize', 'arguments': {'terms': ['Upper gastrointestinal bleeding', 'duodenal ulcer', 'acute blood loss anemia']}}}
{'function': {'name': 'followup_gap', 'arguments': {'appts': [{'date': '2024-09-05', 'provider': 'Andrew Peterson', 'specialty': 'GI', 'time': '2:15 PM'}, {'date': '2024-10-18', 'provider': '', 'specialty': '', 'time': '11:00 AM'}], 'discharge_date': '2024-08-25'}}}
{ "ready": false, "reasons": ["Abnormal labs: Hgb low, Hct low, BUN high", "Follow-up appointment >7 days"], "summary": { "labs": { "abnormal": [ { "name_input": "Hgb", "name_norm": "hemoglobin", "value": "10.2", "value_num": 10.2, "ref_range": "12.0-

# 4. Run tool use evaluation
Demo code for [New blog]()

In [ ]:
from discharge_agent.evaluation.evaluate_tool_use import run_tool_eval

run_tool_eval("data/processed_notes/tool_evaluation_samples.jsonl")


=== Tool Use Evaluation ===
Tool              Should    Used    Use%      OK     OK%    p50 ms    p95 ms
flag_labs              9       1     11%       1    100%       0.2       0.2
followup_gap          10       8     80%       8    100%       0.3       0.6
umls_normalize        10       9     90%       9    100%     371.5     390.3


In [1]:
from discharge_agent.evaluation.evaluate_tool_use import run_tool_eval

discharge_eval_results = run_tool_eval("data/processed_notes/tool_evaluation_samples_v2.jsonl")


=== Tool Use Evaluation ===
Tool              Should    Used    Use%      OK     OK%    p50 ms    p95 ms
flag_labs              9      10    111%      10    100%       0.1       0.2
followup_gap          10      10    100%      10    100%       0.3       0.5
umls_normalize        10      10    100%      10    100%     752.3    2307.8


In [ ]:
json.loads(discharge_eval_results[0])

{'ready': False,
 'reasons': ['Hemoglobin low (9.2) indicating anemia.',
  'Scheduled follow-up is 14 days after discharge, exceeding 7-day guideline.'],
 'summary': {'labs': {'abnormal': [{'name_input': 'Hemoglobin',
     'name_norm': 'hemoglobin',
     'value': '9.2',
     'value_num': 9.2,
     'ref_range': '12.0-16.0',
     'status': 'low'}],
   'ok': False},
  'followup': {'days_to_earliest_followup': 14, 'meets_goal': False},
  'meds': {'new_medications': [{'name': 'Oxycodone',
     'dose': '5mg',
     'frequency': 'Q4H PRN pain'},
    {'name': 'Acetaminophen', 'dose': '650mg', 'frequency': 'Q6H'},
    {'name': 'Docusate', 'dose': '100mg', 'frequency': 'BID'}],
   'dose_changes': [{'name': 'Calcium carbonate',
     'old_dose': '500mg',
     'new_dose': '1200mg',
     'frequency': 'daily'},
    {'name': 'Vitamin D3',
     'old_dose': '1000u',
     'new_dose': '2000u',
     'frequency': 'daily'}]},
  'diagnoses': [{'input': 'femoral neck fracture',
    'cui': 'C0015806',
    'pref_